# HyperDrive demo notebook

This notebook will demostrate the use of the AzureML tool HyperDrive which allows for distributed hyperparameter tuning.

 Hyperparameters are used to control the training of a machine learning model and hyperparameter tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm for the specific application. This requires compiling and training a model over and over with different combinations of hyperparameters in a defined hyperparameter space, so the process can be very time intensive. Parallelising this processing can significantly increase the time efficiency of this process. 

The environment that this notebook works best with is: <b>Python 3.8 - AzureML </b>

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
import prd_pipeline

## Set up azure experiment


In [5]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment
from azureml.core import Experiment, ComputeTarget, ScriptRunConfig

In [6]:
prd_ws = Workspace.from_config()

In [7]:
# azure_dataset_name = 'prd_merged_202008_storm_francis' #'sd3'
azure_experiment_name='prd_fraction_models'
azure_env_name = 'prd_ml_cluster'
cluster_name = 'prd-ml-fractions-cluster'

In [8]:
prd_model_name = 'azml_cluster_demo_202208'

In [9]:
target_parameter = [
    'radar_fraction_in_band_instant_0.25',
    'radar_fraction_in_band_instant_2.5',
    'radar_fraction_in_band_instant_7.0',
    'radar_fraction_in_band_instant_10.0'
]
# target_parameter = 'radar_mean_rain_instant'
profile_features = ['air_temperature', 'relative_humidity']
single_lvl_features = [] #'air_pressure_at_sea_level'

In [10]:
feature_dict = {
    'profile': profile_features,
    'single_level': single_lvl_features,
    'target': target_parameter,
} 

In [11]:
prd_exp = Experiment(workspace=prd_ws, name=azure_experiment_name)
prd_exp

Name,Workspace,Report Page,Docs Page
prd_fraction_models,precip_rediagnosis,Link to Azure Machine Learning studio,Link to Documentation


Get the AzML environment (basically a conda environment) from the workspace.

In [12]:
# test_run = azureml.core.get_run(prd_exp, 'HD_18fd3ffb-6021-4845-88d0-e706c643a9fa')

In [13]:
# test_run.get_best_run_by_primary_metric()

In [14]:
prd_env = Environment.get(workspace=prd_ws, name=azure_env_name)
prd_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04",
        "baseImageRegistry": {
            "address": "mcr.microsoft.com",
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {},
    "inferencingStackVersion": null,
    "name": "prd_ml_cluster",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "dependencies": [
                "python=3.8",

### Load data

Use prd_pipeline to load and preprocess data

In [15]:
# import importlib 
# importlib.reload(prd_cluster_train_demo)

In [16]:
azure_dataset_name = 'prd_merged_all_events_files'
train202208_dataset_all = azureml.core.Dataset.get_by_name(prd_ws, name=azure_dataset_name)
prd_prefix = 'prd'
merged_prefix = prd_prefix + '_merged'
csv_file_suffix = 'csv'

In [17]:
# with train202208_dataset_all.mount() as mount1:
#     print('loading all data')
#     prd_path_list = [p1 for p1 in pathlib.Path(mount1.mount_point).rglob('*csv') ]
#     merged_df = pd.concat([pd.read_csv(p1) for p1 in prd_path_list])

In [18]:
# test_fraction=0.2
# df_train, df_test = prd_pipeline.random_time_space_sample(
#     merged_df, test_fraction=test_fraction, random_state=np.random.RandomState(), sampling_columns = ['time', 'latitude', 'longitude'])
# data_splits, data_dims_dict = prd_pipeline.preprocess_data(
#     df_train, feature_dict, test_fraction=test_fraction/(1-test_fraction))

In [19]:
# input_data = prd_pipeline.load_data(
#     prd_ws,
#     dataset_name=azure_dataset_name
# )

In [20]:
# data_splits, data_dims = prd_pipeline.preprocess_data(
#     input_data,
#     test_fraction=0.2,
#     feature_dict={'profile': profile_features, 'single_level': single_lvl_features,'target': target_parameter,},
# )

In [21]:
# these are example calls to the code for easier debugging than running on a separate cluster
# model = prd_cluster_train_demo.build_model(**data_dims)
# model = prd_cluster_train_demo.train_model(model, data_splits)

In [22]:
# model = prd_pipeline.build_model(**data_dims_dict)
# hyperparameter_dict = {
#         'epochs': 1, 
#         'learning_rate': 0.001, 
#         'batch_size': 100
#     }
    
# model = prd_pipeline.train_model(model, data_splits, hyperparameter_dict)

In [23]:
import datetime
log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

### Execute our training run on a cluster with hyperdrive for parallelised hyperparameter tuning

In [24]:
from azureml.train.hyperdrive import RandomParameterSampling, GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

In [25]:
prd_demo_compute_target = ComputeTarget(workspace=prd_ws, name=cluster_name)
prd_demo_compute_target

AmlCompute(workspace=Workspace.create(name='precip_rediagnosis', subscription_id='07efdc52-cd27-48ed-9443-3aad2b6b777b', resource_group='precip_rediagnosis'), name=prd-ml-fractions-cluster, id=/subscriptions/07efdc52-cd27-48ed-9443-3aad2b6b777b/resourceGroups/precip_rediagnosis/providers/Microsoft.MachineLearningServices/workspaces/precip_rediagnosis/computes/prd-ml-fractions-cluster, type=AmlCompute, provisioning_state=Succeeded, location=uksouth, tags={})

Hyperparameters that we want to vary using hyperdrive need to be input arguments for the prd_cluster_train_demo.py script which is called through ScriptRunConfig. Hyperparameters set in prd_demo_args will be overwritten by Hyperdrive. 

In [26]:
nepochs = 1

In [27]:
prd_demo_args = ['--dataset-name', azure_dataset_name,
                 '--model-name', prd_model_name, 
                 '--test-fraction', 0.2,
                ]
prd_demo_args += ['--target-parameter']
prd_demo_args += target_parameter
prd_demo_args += ['--profile-features']
prd_demo_args += profile_features
prd_demo_args += ['--single-level_features']
prd_demo_args += single_lvl_features
prd_demo_args += ['--epochs', nepochs]
prd_demo_args += ['--batch-size', 128]
prd_demo_args += ['--learning-rate', 0.01]

prd_demo_args

['--dataset-name',
 'prd_merged_all_events_files',
 '--model-name',
 'azml_cluster_demo_202208',
 '--test-fraction',
 0.2,
 '--target-parameter',
 'radar_fraction_in_band_instant_0.25',
 'radar_fraction_in_band_instant_2.5',
 'radar_fraction_in_band_instant_7.0',
 'radar_fraction_in_band_instant_10.0',
 '--profile-features',
 'air_temperature',
 'relative_humidity',
 '--single-level_features',
 '--epochs',
 1,
 '--batch-size',
 128,
 '--learning-rate',
 0.01]

In [28]:
prd_run_src = ScriptRunConfig(
    source_directory=os.getcwd(),
    script='prd_cluster_train_demo.py',
    arguments=prd_demo_args,
    compute_target=prd_demo_compute_target,
    environment=prd_env
)

### Run on cluster

In [29]:
# prd_run = prd_exp.submit(prd_run_src)
# prd_run

### HyperDrive configuration

The next step is to configure our HyperDrive run. The run config defined above is passed to our HyperDriveConfig as the run_config. We must provide a <code>hyperparameter_sampling</code> explained in more detail below. We are required to also provide the <code>primary_metric_name</code> (either be the models loss function or a metric define when compiling the model) and the <code>primary_metric_goal</code> to either minimize or maximize this metric. We must also define <code>max_total_runs</code> (the upper bound of the number of runs, may be smaller depending on the defined hyperparameter space and sampling strategy) and <code>max_concurrent_runs</code>, if this is set to None all run are launched in parallel.
There is also the option to define an early stopping policy with the <code>policy</code> argument.

#### Hyperparameter sampling

There are three different classes for sampling the hyperparameter space: 
- <code>GridParameterSampling</code>: define a search space as a grid of hyperparameter based on the given hyperparameter space, then evaluates every position in the grid in order (note if max_total_runs < total potential combinations, then this will only run a subsample of the grid).
- <code>RandomParameterSampling</code>: randomly samples hyperparameter combinations from the hyperparameter space 
- <code>BayesianParameterSampling</code>: defines Bayesian sampling over a hyperparameter space, tries to intelligently pick the next sample of hyperparameters based on how the previous samples performed.

We then define the hyperparameter space from which to select the combination of hyperparameters to assess. The hyperparameters that we want to tune should be input arguments to the script which is being run by ScriptRunConfig and any hyperparameter arguments that are input will be overwritten by hyperdrive.

In AzureML, there are different ways to define the set to sample each hyperparameters from (<code>choice</code>, <code>lognormal</code>, <code>loguniform</code>, <code>normal</code>, <code>qlognormal</code>, <code>qloguniform</code>, <code>qnormal</code>, <code>quniform</code>, <code>randint</code> and <code>uniform</code>).

In [42]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(32, 64, 128),
        '--learning-rate': choice(0.1, 0.01, 0.001, 0.0001),
        '--epochs': choice(10, 20, 50, 100)
    }
)

#### Early stopping policy

We can define an early stopping policy in which means that any poorly performing experiment runs are canceled and new ones started. Here we use BanditPolicy with a slack criteria of 0.1 (the ratio of slack allowed with respect to the best performing training run) and evaluation internal of 2 (the frequency for applying the policy, here every two training steps).

In [43]:
early_stop_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [44]:
htc = HyperDriveConfig(run_config=prd_run_src, 
                       hyperparameter_sampling=ps, 
                       policy=early_stop_policy, 
                       primary_metric_name='val_loss', 
                       primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                       max_total_runs=18,
                       max_concurrent_runs=12)

### HyperDrive running

In [45]:
prd_run = prd_exp.submit(htc)
prd_run

Experiment,Id,Type,Status,Details Page,Docs Page
prd_fraction_models,HD_2d856d18-157c-4abc-9e10-4f8fa8d038a7,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [40]:
prd_run.wait_for_completion()
assert(prd_run.get_status() == "Completed")

### HyperDrive results

We can then select the model which performs best against the selected primary metric, as defined within the HyperDriveConfig.

In [39]:
prd_best_run = prd_run.get_best_run_by_primary_metric()
prd_best_run

We can also return information from each of the different hyperdrive child run

In [40]:
prd_run.get_children_sorted_by_primary_metric()

[{'run_id': 'HD_baf8cd1f-a629-4487-8eff-b3583b14ab42_0',
  'hyperparameters': '{"--batch-size": 64, "--learning-rate": 0.12361085064204563}',
  'best_primary_metric': None,
  'status': 'Completed'},
 {'run_id': 'HD_baf8cd1f-a629-4487-8eff-b3583b14ab42_3',
  'hyperparameters': '{"--batch-size": 32, "--learning-rate": 0.03355127495789812}',
  'best_primary_metric': None,
  'status': 'Completed'},
 {'run_id': 'HD_baf8cd1f-a629-4487-8eff-b3583b14ab42_2',
  'hyperparameters': '{"--batch-size": 64, "--learning-rate": 0.00935085121731626}',
  'best_primary_metric': None,
  'status': 'Completed'},
 {'run_id': 'HD_baf8cd1f-a629-4487-8eff-b3583b14ab42_1',
  'hyperparameters': '{"--batch-size": 128, "--learning-rate": 0.0069036924601328115}',
  'best_primary_metric': None,
  'status': 'Completed'}]

In [37]:
prd_run.get_metrics()

{}

In [38]:
test_run.get_metrics()

NameError: name 'test_run' is not defined

In [ ]:
prd_run.get_hyperparameters()

In [ ]:
test_run.get_hyperparameters()